In [1]:
import sys
sys.path.append('/media/raid/graham/compute/compute/CellHier/graham/cellhier/')
from cellhier.general import *
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
%matplotlib inline
%pylab inline
from matplotlib import pyplot as plt
import skimage
import pickle
import seaborn as sns
import networkx as nx
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components
import itertools
from skimage.color import label2rgb
from scipy import ndimage as ndi
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.csgraph import connected_components as cc
import random
import copy
from tqdm.notebook import tqdm
from multiprocessing import Pool
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
#get CRC dataset
cells2 = pd.read_pickle('../submissiondata/cells2_salil')

In [3]:
# need this to deal with CN1 called dirt in original dataset
cn_names = {i:i if i!=0 else i+1 for i in range(10)}

In [4]:
# make some dicts
spot_ids  = cells2['spots'].unique()
pat_gp = cells2[['patients','groups']].drop_duplicates()
pat_to_gp= {a:b for a,b in pat_gp.values}
spot_to_patient = {a:b for a,b in cells2[['spots','patients']].drop_duplicates().values}

In [5]:
def segment_instances(spot_id, num_neighbors,min_instance_size,return_kgr = False):
    '''
    This takes a given TMA spot and segments it using connected components of
    k-neighbors graph computed with num_neighbors; 
    only the cells residing in instances with size greater than min_instance_size are returned
    '''
    
    spot = cells2[cells2['spots']==spot_id]
    nn = NearestNeighbors(n_neighbors=10).fit(spot[['X:X','Y:Y']])
    kgr = nn.kneighbors_graph()


    min_instance_size = 5
    spot_cn_cell_idxs = {}
    spot_inst_assignments = {}
    instance_assignments = {}
    good_instances = {}

    for cn in range(10):        
        idx = np.where(spot['neighborhood10']==cn)[0]
        spot_cn_cell_idxs[cn] = idx
        instance_assignments[cn] = cc(kgr[idx,:][:,idx])[1]
        counts = np.unique(instance_assignments[cn],return_counts=True)
        good_instances[cn] = counts[0][counts[1]>min_instance_size]
        cn_good_inst_idx = np.where(np.isin(instance_assignments[cn],good_instances[cn]))[0]
        good_inst_assignments = instance_assignments[cn][cn_good_inst_idx]
        spot_good_inst_cell_idx = spot_cn_cell_idxs[cn][cn_good_inst_idx]
        
        # this is the pair of index (in the spot) of good cells, as well as their assignments
        spot_inst_assignments[cn] = (spot_good_inst_cell_idx,good_inst_assignments)
    
    if return_kgr:
        return spot, spot_cn_cell_idxs, spot_inst_assignments, kgr
    
    return spot, spot_cn_cell_idxs, spot_inst_assignments

In [6]:
# construct the adj graphs
num_neighbors = 10
min_instance_size = 5

spot_adj_graphs = {}
spot_data = {}


for spot_id in spot_ids:

    spot_data[spot_id] = segment_instances(spot_id,num_neighbors=num_neighbors,min_instance_size=min_instance_size,return_kgr = True)
    spot, spot_cn_cell_idxs, inst_assignments,kgr  = spot_data[spot_id]

    for cn1 in range(10):
        for cn2 in range(cn1):
            e1,e2 = kgr[inst_assignments[cn1][0],:][:,inst_assignments[cn2][0]].nonzero()
            for s,t in set(list(zip(inst_assignments[cn1][1][e1],inst_assignments[cn2][1][e2]))):
                spot_adj_graphs.setdefault(spot_id, nx.Graph()).add_edge( (spot_id,cn1,s), (spot_id,cn2,t))
    



In [7]:
neighbors = {}
colors = {}
node_to_idx = {}
idx_to_node = {}
for spot in spot_ids:    
    graph = spot_adj_graphs[spot].copy()
    neighbors[spot] = {}
    colors[spot] = {}
    node_to_idx[spot] = {}
    idx_to_node[spot] = {}
    for i,node in enumerate(graph.nodes()):
        colors[spot][i] = node[1]    
        node_to_idx[spot][node] = i
        idx_to_node[spot][i] = node

    for i,node in idx_to_node[spot].items():
        neighbors[spot][i] = [node_to_idx[spot][t] for _,t in graph.edges(node)]
        
relabeled_spot_adj_graphs = {spot_id: nx.relabel_nodes(spot_adj_graphs[spot_id], node_to_idx[spot_id]) for spot_id in spot_ids}

In [8]:
import colormc

In [9]:
t = time.time()
num_samples = 20000
a = colormc.ColorMC(neighbors['46_B'],colors['46_B'])
samps = a.runMC(num_samples,1,0)
s = time.time()-t
print(1000*s/num_samples,'ms per step')

0.04287745952606201 ms per step


In [10]:
t = time.time()
num_samples = 20000
a = colormc.ColorMCSet(neighbors['46_B'],colors['46_B'])
samps = a.runMC(num_samples,1,0)
s = time.time()-t
print(1000*s/num_samples,'ms per step')

0.05173041820526123 ms per step


In [13]:
def cpp_mix_spot_graph(input_data):
    neighbors = input_data['neighbors']
    colors = input_data['colors']
    num_samples = input_data['num_samples']
    sample_every = input_data['sample_every']
    burn_steps = input_data['burn_steps']
    isom = input_data['isom']
    if isom=='set':
        cmc = colormc.ColorMCSet(neighbors, colors)
    if isom=='exact':
        cmc = colormc.ColorMC(neighbors, colors)
        
    samps = cmc.runMC(num_samples*sample_every + burn_steps, sample_every, burn_steps)
    return samps


In [ ]:
num_samples = 100000
sample_every = 2000
burn_steps = 100000
exact_input_datas = []
set_input_datas = []
spots = spot_ids
for spot in spots:
    exact_input_datas.append({'isom':'set','colors': colors[spot], 'neighbors': neighbors[spot], 'num_samples': num_samples, 'sample_every': sample_every, 'burn_steps': burn_steps})
    set_input_datas.append({'isom':'exact','colors': colors[spot], 'neighbors': neighbors[spot], 'num_samples': num_samples, 'sample_every': sample_every, 'burn_steps': burn_steps})
    
pool = Pool(64)
set_samples = list(tqdm(pool.imap(cpp_mix_spot_graph,set_input_datas), total = len(spots)))
print('set samples done')
exact_samples = list(tqdm(pool.imap(cpp_mix_spot_graph,exact_input_datas), total = len(spots)))
pool.close()


no viable transpositions at step 0
no viable transpositions at step 0


# !!!!! There was a typo above where the exact input datas have isom=set and vice versa !!!!!

In [18]:
data = {'set_samples': {spot:samp for spot,samp in zip(spot_ids,exact_samples)}, 'exact_samples':{spot:samp for spot,samp in zip(spot_ids,set_samples)}}

In [25]:
with open('big_MH_data', 'wb') as file:
    pickle.dump(data, file)

In [16]:
col_inputs = []
col_keys = []
for spot, col_list in zip(spots,samples):
    for i,col in enumerate(col_list):
        col_keys.append((spot,i))
        graph = relabeled_spot_adj_graphs[spot]
        col_inputs.append({'graph':graph,'col':col})


140


In [17]:
print(len(exact_samples))

140
